# Various Recommender models on Retail data

## Environment Setup

In [ ]:
import tensorflow as tf

## loading packages
import sys
import random
import datetime
import numpy as np
import pandas as pd
from math import ceil
from tqdm import trange
from subprocess import call
from itertools import islice
from sklearn.metrics import roc_auc_score
from sklearn import preprocessing
from sklearn.preprocessing import normalize
from sklearn.neighbors import NearestNeighbors
from scipy.sparse import csr_matrix, dok_matrix
from sklearn.model_selection import ParameterGrid

import matplotlib.pyplot as plt
import seaborn as sns

import math
import pandas as pd
import numpy as np
import os
import matplotlib.pyplot as plt
import matplotlib.style as style
import seaborn as sns
import heapq ## for retrieval topK
import multiprocessing
import pickle
import matplotlib.pyplot as plt
import seaborn as sns
import random
from datetime import datetime
from pprint import pprint 
from time import time
from scipy.sparse.linalg import svds, eigs

from functools import wraps
from sklearn.metrics import (
    mean_squared_error,
    mean_absolute_error,
    r2_score,
    explained_variance_score,
    roc_auc_score,
    log_loss,
)

import warnings
warnings.filterwarnings('ignore')

import numpy as np
import os
import pandas as pd
import scipy.sparse
import time
import sys

from fastai.collab import *
from fastai.tabular import *
from fastai.text import *

!pip install git+https://github.com/maciejkula/spotlight.git@master#egg=spotlight

!git clone https://github.com/microsoft/recommenders.git
sys.path.append('/content/recommenders/')

In [ ]:
## loading data
file_path =  '/content/drive/My Drive/Recommendation/'
file_name = 'rawdata.csv'
df = pd.read_csv(file_path+file_name, header = 0,
                 names = ['event','userid','itemid','timestamp'],
                 dtype={0:'category', 1:'category', 2:'category'},
                 parse_dates=['timestamp'])

## dropping exact duplicates
## df = df.drop_duplicates()

## userid normalization
userid_encoder = preprocessing.LabelEncoder()
df.userid = userid_encoder.fit_transform(df.userid)

## itemid normalization
itemid_encoder = preprocessing.LabelEncoder()
df.itemid = itemid_encoder.fit_transform(df.itemid)

df.head()

,event,userid,itemid,timestamp
0,view_item,3141,236,2020-01-13 16:05:31.244000+00:00
1,add_to_cart,3421,1001,2020-01-13 22:36:38.680000+00:00
2,view_item,550,972,2020-01-14 10:54:41.886000+00:00
3,view_item,550,972,2020-01-14 10:54:47.692000+00:00
4,add_to_cart,550,972,2020-01-14 10:54:48.479000+00:00


In [ ]:
df.info()
df.shape[0]/df.userid.nunique()
df.describe().T
df.describe(exclude='int').T
df.userid.cat.codes
df.event.value_counts()/df.userid.nunique()
df.timestamp.max() - df.timestamp.min()

In [ ]:
grouped_df = df.groupby(['userid', 'itemid'])['event'].sum().reset_index()

## Data Transformation
- Count
- Weighted Count
- Time dependent Count
- Negative Sampling

### A. Count

In [ ]:
data_count = df.groupby(['userid', 'itemid']).agg({'timestamp': 'count'}).reset_index()
data_count.columns = ['userid', 'itemid', 'affinity']
data_count.head()

,userid,itemid,affinity
0,0,328,1
1,1,1122,1
2,1,1204,1
3,1,1271,1
4,1,1821,1


### B. Weighted Count

In [ ]:
data_w = df.loc[df.event!='remove_from_cart',:]

In [ ]:
affinity_weights = {
    'view_item': 1,
    'add_to_cart': 3, 
    'begin_checkout': 5, 
    'purchase': 6,
    'remove_from_cart': 3
}

In [ ]:
data_w['weight'] = data_w['event'].apply(lambda x: affinity_weights[x])

In [ ]:
data_wcount = data_w.groupby(['userid', 'itemid'])['weight'].sum().reset_index()
data_wcount.columns = ['userid', 'itemid', 'affinity']
data_wcount.head()

,userid,itemid,affinity
0,0,328,6
1,1,1122,6
2,1,1204,6
3,1,1271,6
4,1,1821,6


### C. Time dependent Count

In [ ]:
T = 30
t_ref = datetime.utcnow()

In [ ]:
data_w['timedecay'] = data_w.apply(
    lambda x: x['weight'] * math.exp(-math.log2((t_ref - pd.to_datetime(x['timestamp']).tz_convert(None)).days / T)), 
    axis=1
)

In [ ]:
data_w.head()

,event,userid,itemid,timestamp,weight,timedecay
0,view_item,3141,236,2020-01-13 16:05:31.244000+00:00,1,0.307103
1,add_to_cart,3421,1001,2020-01-13 22:36:38.680000+00:00,3,0.921309
2,view_item,550,972,2020-01-14 10:54:41.886000+00:00,1,0.307103
3,view_item,550,972,2020-01-14 10:54:47.692000+00:00,1,0.307103
4,add_to_cart,550,972,2020-01-14 10:54:48.479000+00:00,3,0.921309


In [ ]:
data_wt = data_w.groupby(['userid', 'itemid'])['timedecay'].sum().reset_index()
data_wt.columns = ['userid', 'itemid', 'affinity']
data_wt.head()

,userid,itemid,affinity
0,0,328,2.261452
1,1,1122,2.713421
2,1,1204,2.713421
3,1,1271,2.713421
4,1,1821,2.713421


## Negative Sampling

In [ ]:
data_b = df[['userid', 'itemid']].copy()
data_b['feedback'] = 1
data_b = data_b.drop_duplicates()

In [ ]:
users = df['userid'].unique()
items = df['itemid'].unique()

In [ ]:
interaction_lst = []
for user in users:
    for item in items:
        interaction_lst.append([user, item, 0])

data_all = pd.DataFrame(data=interaction_lst, columns=["userid", "itemid", "feedbackAll"])

In [ ]:
data_ns = pd.merge(data_all, data_b, on=['userid', 'itemid'], how='outer').fillna(0).drop('feedbackAll', axis=1)
data_ns.head()

,userid,itemid,feedback
0,3141,236,1.0
1,3141,1001,0.0
2,3141,972,0.0
3,3141,286,0.0
4,3141,1002,0.0


## Other

In [ ]:
dfx = df[['userid','itemid','eventStrength','timestamp']]
dfx.head()

,userid,itemid,eventStrength,timestamp
0,3141,236,1.0,2020-01-13 16:05:31.244000+00:00
1,3421,1001,3.0,2020-01-13 22:36:38.680000+00:00
2,550,972,1.0,2020-01-14 10:54:41.886000+00:00
3,550,972,1.0,2020-01-14 10:54:47.692000+00:00
4,550,972,3.0,2020-01-14 10:54:48.479000+00:00


## Train Test Split

In [ ]:
data = data_w[['userid','itemid','timedecay','timestamp']]

In [ ]:
col = {
  'col_user': 'userid',
  'col_item': 'itemid',
  'col_rating': 'timedecay',
  'col_timestamp': 'timestamp',
}

col3 = {
  'col_user': 'userid',
  'col_item': 'itemid',
  'col_timestamp': 'timestamp',
}

In [ ]:
from reco_utils.dataset.python_splitters import python_chrono_split
train, test = python_chrono_split(data, ratio=0.75, min_rating=10, 
                                  filter_by='user', **col3)

In [ ]:
train.head()

,userid,itemid,timedecay,timestamp
16679,7,1464,0.320618,2020-01-16 06:42:31.341000+00:00
16691,7,1464,0.320618,2020-01-16 06:43:29.482000+00:00
16692,7,2109,0.320618,2020-01-16 06:43:42.262000+00:00
16694,7,1464,0.320618,2020-01-16 06:43:57.961000+00:00
16805,7,201,0.320618,2020-01-16 06:45:55.261000+00:00


In [ ]:
train.loc[train.userid==7,:]

,userid,itemid,timedecay,timestamp
16679,7,1464,0.320618,2020-01-16 06:42:31.341000+00:00
16691,7,1464,0.320618,2020-01-16 06:43:29.482000+00:00
16692,7,2109,0.320618,2020-01-16 06:43:42.262000+00:00
16694,7,1464,0.320618,2020-01-16 06:43:57.961000+00:00
16805,7,201,0.320618,2020-01-16 06:45:55.261000+00:00
16890,7,2570,0.320618,2020-01-16 06:54:12.315000+00:00
16999,7,2570,0.320618,2020-01-16 06:54:29.130000+00:00
17000,7,2570,0.961855,2020-01-16 06:54:35.097000+00:00


In [ ]:
test.loc[test.userid==7,:]

,userid,itemid,timedecay,timestamp
17001,7,1464,0.320618,2020-01-16 06:54:41.415000+00:00
17003,7,1464,0.961855,2020-01-16 06:54:44.195000+00:00


## Baseline

In [ ]:
## Recommending the most popular items is intuitive and simple approach
item_counts = train['itemid'].value_counts().to_frame().reset_index()
item_counts.columns = ['itemid', 'count']
item_counts.head()

,itemid,count
0,2564,466
1,1463,300
2,1710,270
3,1985,243
4,886,235


In [ ]:
user_item_col = ['userid', 'itemid']

## Cross join users and items
test_users = test['userid'].unique()
user_item_list = list(itertools.product(test_users, item_counts['itemid']))
users_items = pd.DataFrame(user_item_list, columns=user_item_col)

print("Number of user-item pairs:", len(users_items))

## Remove seen items (items in the train set) as we will not recommend those again to the users
from reco_utils.dataset.pandas_df_utils import filter_by
users_items_remove_seen = filter_by(users_items, train, user_item_col)

print("After remove seen items:", len(users_items_remove_seen))

Number of user-item pairs: 4137750
After remove seen items: 4120954


In [ ]:
## Generate recommendations
baseline_recommendations = pd.merge(item_counts, users_items_remove_seen, 
                                    on=['itemid'], how='inner')
baseline_recommendations.head()

,itemid,count,userid
0,2564,466,7
1,2564,466,21
2,2564,466,73
3,2564,466,75
4,2564,466,113


In [ ]:
from reco_utils.evaluation.python_evaluation import map_at_k
from reco_utils.evaluation.python_evaluation import precision_at_k
from reco_utils.evaluation.python_evaluation import ndcg_at_k 
from reco_utils.evaluation.python_evaluation import recall_at_k
from reco_utils.evaluation.python_evaluation import get_top_k_items

In [ ]:
k = 10

cols = {
  'col_user': 'userid',
  'col_item': 'itemid',
  'col_rating': 'timedecay',
  'col_prediction': 'count',
}

eval_map = map_at_k(test, baseline_recommendations, k=k, **cols)
eval_ndcg = ndcg_at_k(test, baseline_recommendations, k=k, **cols)
eval_precision = precision_at_k(test, baseline_recommendations, k=k, **cols)
eval_recall = recall_at_k(test, baseline_recommendations, k=k, **cols)

print("MAP:\t%f" % eval_map,
      "NDCG@K:\t%f" % eval_ndcg,
      "Precision@K:\t%f" % eval_precision,
      "Recall@K:\t%f" % eval_recall, sep='\n')

MAP:	0.005391
NDCG@K:	0.010234
Precision@K:	0.006797
Recall@K:	0.011208


In [ ]:
from reco_utils.common.notebook_utils import is_jupyter
if is_jupyter():
    ## Record results with papermill for unit-tests
    import papermill as pm
    pm.record("map", eval_map)
    pm.record("ndcg", eval_ndcg)
    pm.record("precision", eval_precision)
    pm.record("recall", eval_recall)

## Model 1 - BPR

In [ ]:
!pip install cornac
import cornac
from reco_utils.recommender.cornac.cornac_utils import predict_ranking

     |████████████████████████████████| 8.1MB 7.6MB/s 


In [ ]:
train.head()

,userid,itemid,timedecay,timestamp
16679,7,1464,0.320618,2020-01-16 06:42:31.341000+00:00
16691,7,1464,0.320618,2020-01-16 06:43:29.482000+00:00
16692,7,2109,0.320618,2020-01-16 06:43:42.262000+00:00
16694,7,1464,0.320618,2020-01-16 06:43:57.961000+00:00
16805,7,201,0.320618,2020-01-16 06:45:55.261000+00:00


In [ ]:
TOP_K = 10
NUM_FACTORS = 200
NUM_EPOCHS = 100
SEED = 40

In [ ]:
train_set = cornac.data.Dataset.from_uir(train.itertuples(index=False), seed=SEED)
print('Number of users: {}'.format(train_set.num_users))
print('Number of items: {}'.format(train_set.num_items))

Number of users: 1839
Number of items: 2250


In [ ]:
bpr = cornac.models.BPR(
    k=NUM_FACTORS,
    max_iter=NUM_EPOCHS,
    learning_rate=0.01,
    lambda_reg=0.001,
    verbose=True,
    seed=SEED
)

In [ ]:
from reco_utils.common.timer import Timer
with Timer() as t:
    bpr.fit(train_set)
print("Took {} seconds for training.".format(t))

100%|██████████| 100/100 [00:01<00:00, 53.85it/s, correct=78.75%, skipped=0.74%]

Optimization finished!
Took 1.8820 seconds for training.


In [ ]:
with Timer() as t:
    all_predictions = predict_ranking(bpr, train, usercol='userid', itemcol='itemid', remove_seen=True)
print("Took {} seconds for prediction.".format(t))

Took 3.4403 seconds for prediction.


In [ ]:
all_predictions.head()

,userid,itemid,prediction
51626,7,2551,-0.446921
51627,7,481,2.553658
51628,7,1185,2.362493
51629,7,1766,1.140463
51630,7,1359,2.135327


In [ ]:
k = 10
cols = {
  'col_user': 'userid',
  'col_item': 'itemid',
  'col_rating': 'timedecay',
  'col_prediction': 'prediction',
}

eval_map = map_at_k(test, all_predictions, k=k, **cols)
eval_ndcg = ndcg_at_k(test, all_predictions, k=k, **cols)
eval_precision = precision_at_k(test, all_predictions, k=k, **cols)
eval_recall = recall_at_k(test, all_predictions, k=k, **cols)

print("MAP:\t%f" % eval_map,
      "NDCG:\t%f" % eval_ndcg,
      "Precision@K:\t%f" % eval_precision,
      "Recall@K:\t%f" % eval_recall, sep='\n')

MAP:	0.004634
NDCG:	0.009463
Precision@K:	0.006688
Recall@K:	0.010223


In [ ]:
## Record results with papermill for tests
pm.record("map", eval_map)
pm.record("ndcg", eval_ndcg)
pm.record("precision", eval_precision)
pm.record("recall", eval_recall)

## NCF

In [ ]:
TOP_K = 10
EPOCHS = 20
BATCH_SIZE = 256
SEED = 42

In [ ]:
from reco_utils.recommender.ncf.ncf_singlenode import NCF
from reco_utils.recommender.ncf.dataset import Dataset as NCFDataset

In [ ]:
cols = {
  'col_user': 'userid',
  'col_item': 'itemid',
  'col_rating': 'timedecay',
  'col_timestamp': 'timestamp',
}

data = NCFDataset(train=train, test=test, seed=SEED, **cols)

In [ ]:
model = NCF (
    n_users=data.n_users, 
    n_items=data.n_items,
    model_type="NeuMF",
    n_factors=4,
    layer_sizes=[16,8,4],
    n_epochs=EPOCHS,
    batch_size=BATCH_SIZE,
    learning_rate=1e-3,
    verbose=10,
    seed=SEED
)

In [ ]:
with Timer() as t:
    model.fit(data)
print("Took {} seconds for training.".format(t))

Took 111.4200 seconds for training.


In [ ]:
users, items, preds = [], [], []
item = list(train.itemid.unique())
for user in train.userid.unique():
    user = [user] * len(item) 
    users.extend(user)
    items.extend(item)
    preds.extend(list(model.predict(user, item, is_list=True)))
all_predictions = pd.DataFrame(data={'userid': users, 'itemid':items, "prediction":preds})
merged = pd.merge(train, all_predictions, on=['userid','itemid'], how="outer")
all_predictions = merged[merged[col['col_rating']].isnull()].drop(col['col_rating'], axis=1)

In [ ]:
k = 10
cols = {
  'col_user': 'userid',
  'col_item': 'itemid',
  'col_rating': 'timedecay',
  'col_prediction': 'prediction',
}

eval_map = map_at_k(test, all_predictions, k=k, **cols)
eval_ndcg = ndcg_at_k(test, all_predictions, k=k, **cols)
eval_precision = precision_at_k(test, all_predictions, k=k, **cols)
eval_recall = recall_at_k(test, all_predictions, k=k, **cols)

print("NDCG:\t%f" % eval_ndcg,
      "Precision@K:\t%f" % eval_precision,
      "Recall@K:\t%f" % eval_recall, sep='\n')

NDCG:	0.010389
Precision@K:	0.007558
Recall@K:	0.012055


## Model - SARS

In [ ]:
from reco_utils.recommender.sar.sar_singlenode import SARSingleNode

In [ ]:
TOP_K = 10

In [ ]:
header = {
    "col_user": "userid",
    "col_item": "itemid",
    "col_rating": "timedecay",
    "col_timestamp": "timestamp",
    "col_prediction": "prediction",
}

In [ ]:
SARSingleNode?

In [ ]:
model = SARSingleNode(
    similarity_type="jaccard", 
    time_decay_coefficient=0, 
    time_now=None, 
    timedecay_formula=False, 
    **header
)

In [ ]:
model.fit(train)

In [ ]:
## all ranking metrics have the same arguments
args = [test, top_k]
kwargs = dict(col_user='userid', 
              col_item='itemid', 
              col_rating='timedecay', 
              col_prediction='prediction', 
              relevancy_method='top_k', 
              k=TOP_K)

eval_map = map_at_k(*args, **kwargs)
eval_ndcg = ndcg_at_k(*args, **kwargs)
eval_precision = precision_at_k(*args, **kwargs)
eval_recall = recall_at_k(*args, **kwargs)

print(f"Model:",
      f"Top K:\t\t {TOP_K}",
      f"MAP:\t\t {eval_map:f}",
      f"NDCG:\t\t {eval_ndcg:f}",
      f"Precision@K:\t {eval_precision:f}",
      f"Recall@K:\t {eval_recall:f}", sep='\n')

Model:
Top K:		 10
MAP:		 0.025638
NDCG:		 0.033655
Precision@K:	 0.019195
Recall@K:	 0.035803


In [ ]:
## Instantiate the recommender models to be compared
gmf = cornac.models.GMF(
    num_factors=8,
    num_epochs=10,
    learner="adam",
    batch_size=256,
    lr=0.001,
    num_neg=50,
    seed=123,
)
mlp = cornac.models.MLP(
    layers=[64, 32, 16, 8],
    act_fn="tanh",
    learner="adam",
    num_epochs=10,
    batch_size=256,
    lr=0.001,
    num_neg=50,
    seed=123,
)
neumf1 = cornac.models.NeuMF(
    num_factors=8,
    layers=[64, 32, 16, 8],
    act_fn="tanh",
    learner="adam",
    num_epochs=10,
    batch_size=256,
    lr=0.001,
    num_neg=50,
    seed=123,
)
neumf2 = cornac.models.NeuMF(
    name="NeuMF_pretrained",
    learner="adam",
    num_epochs=10,
    batch_size=256,
    lr=0.001,
    num_neg=50,
    seed=123,
    num_factors=gmf.num_factors,
    layers=mlp.layers,
    act_fn=mlp.act_fn,
).pretrain(gmf, mlp)

In [ ]:
with Timer() as t:
    gmf.fit(train_set)
print("Took {} seconds for training.".format(t))

100%|██████████| 10/10 [02:35<00:00, 15.52s/it, loss=0.124]

Took 155.4467 seconds for training.


In [ ]:
with Timer() as t:
    mlp.fit(train_set)
print("Took {} seconds for training.".format(t))

100%|██████████| 10/10 [02:30<00:00, 15.00s/it, loss=0.0898]

Took 150.4327 seconds for training.


In [ ]:
with Timer() as t:
    neumf1.fit(train_set)
print("Took {} seconds for training.".format(t))

100%|██████████| 10/10 [02:30<00:00, 15.03s/it, loss=0.0857]

Took 150.7486 seconds for training.


In [ ]:
with Timer() as t:
    neumf2.fit(train_set)
print("Took {} seconds for training.".format(t))

100%|██████████| 10/10 [02:36<00:00, 15.66s/it, loss=0.08]

Took 157.4725 seconds for training.


In [ ]:
def rec_eval(model):
  with Timer() as t:
      all_predictions = predict_ranking(model, train, usercol='userid', itemcol='itemid', remove_seen=True)

  k = 10
  cols = {
    'col_user': 'userid',
    'col_item': 'itemid',
    'col_rating': 'timedecay',
    'col_prediction': 'prediction',
  }

  eval_map = map_at_k(test, all_predictions, k=k, **cols)
  eval_ndcg = ndcg_at_k(test, all_predictions, k=k, **cols)
  eval_precision = precision_at_k(test, all_predictions, k=k, **cols)
  eval_recall = recall_at_k(test, all_predictions, k=k, **cols)

  print("MAP:\t%f" % eval_map,
        "NDCG:\t%f" % eval_ndcg,
        "Precision@K:\t%f" % eval_precision,
        "Recall@K:\t%f" % eval_recall, sep='\n')

In [ ]:
rec_eval(gmf);
rec_eval(mlp);
rec_eval(neumf1);
rec_eval(neumf2);

Took 4.5247 seconds for prediction.
MAP:	0.005331
NDCG:	0.010839
Precision@K:	0.007939
Recall@K:	0.012291
Took 4.7216 seconds for prediction.
MAP:	0.005211
NDCG:	0.009625
Precision@K:	0.006525
Recall@K:	0.010316
Took 4.8190 seconds for prediction.
MAP:	0.005261
NDCG:	0.009443
Precision@K:	0.006362
Recall@K:	0.010589
Took 4.8854 seconds for prediction.
MAP:	0.007244
NDCG:	0.014118
Precision@K:	0.009788
Recall@K:	0.014048


## DeepRec Ranking Models

In [ ]:
!git clone https://github.com/cheungdaven/DeepRec.git
sys.path.append('/content/DeepRec/')
## sys.path.append(os.path.join(os.path.dirname(__file__), '..'))

In [ ]:
## from models.item_ranking.cdae import ICDAE
from models.item_ranking.bprmf import BPRMF
from models.item_ranking.cml import CML
from models.item_ranking.neumf import NeuMF
from models.item_ranking.gmf import GMF
from models.item_ranking.jrl import JRL
from models.item_ranking.mlp import MLP
from models.item_ranking.lrml import LRML

In [ ]:
epochs = 10
num_factors = 10
display_step = 1000
batch_size = 256
learning_rate = 1e-3
reg_rate = 0.1

In [ ]:
try:
  gpus = tf.config.experimental.list_physical_devices('GPU')
  tf.config.experimental.set_memory_growth(gpus[0], True)
except:
  pass

In [ ]:
n_users = df.userid.unique().shape[0]
n_items = df.itemid.unique().shape[0]

train_row = []
train_col = []
train_rating = []
for line in train.itertuples():
    train_row.append(line[1])
    train_col.append(line[2])
    train_rating.append(line[3])   
train_matrix = csr_matrix((train_rating, (train_row, train_col)), shape=(n_users, n_items))

test_row = []
test_col = []
test_rating = []
for line in test.itertuples():
    test_row.append(line[1])
    test_col.append(line[2])
    test_rating.append(line[3])
test_matrix = csr_matrix((test_rating, (test_row, test_col)), shape=(n_users, n_items))

test_dict = {}
for u in range(n_users):
    test_dict[u] = test_matrix.getrow(u).nonzero()[1]

In [ ]:
train_data, test_data, n_user, n_item = train_matrix.todok(), test_dict, n_users, n_items

In [ ]:
model = GMF(n_user, n_item)
model.build_network()
model.execute(train_data, test_data)

In [ ]:
model = JRL(n_user, n_item)
model.build_network()
model.execute(train_data, test_data)

In [ ]:
model = MLP(n_user, n_item)
model.build_network()
model.execute(train_data, test_data)

In [ ]:
config = tf.ConfigProto()
config.gpu_options.allow_growth = True

In [ ]:
with tf.Session(config=config) as sess:
  train_data, test_data, n_user, n_item = train_matrix.todok(), test_dict, n_users, n_items
  model = LRML(sess, n_user, n_item, epoch=epochs, batch_size=batch_size)
  model.build_network()
  model.execute(train_data, test_data)

LRML.





Instructions for updating:
Use tf.where in 2.0, which has the same broadcast rule as np.where
Instructions for updating:
Call initializer instance with the dtype argument instead of passing it to the constructor

65
data preparation finished.

Epoch: 0000; ------------------------
precision@10:0.00255573681348559
recall@10:0.007459988534424281
precision@5:0.0033713974986405663
recall@5:0.005255148482928438
map:0.00873635552532191
mrr:0.015335400314334816
ndcg:0.07201765186618744
ndcg@5:0.0054521443401014565
ndcg@10:0.0061940946844342435
Epoch: 0005; ------------------------
precision@10:0.0022838499184339315
recall@10:0.007641868427944132
precision@5:0.0020663404023926048
recall@5:0.00446347652709806
map:0.008066203867174779
mrr:0.012507585616236624
ndcg:0.07297931579895377
ndcg@5:0.003951060806305347
ndcg@10:0.005420745496276038
